In [81]:
#Player stats
# Get all player stats for each year
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog, playercareerstats, playercareerstats, playergamelogs
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamgamelogs, leaguegamelog, boxscoreadvancedv2


import numpy as np
import pandas as pd
import os
import time


13    1.8
Name: NET_RATING, dtype: float64


In [71]:

def get_team_for_year(start_year):
    season = '{}-{}'.format(start_year, str(start_year + 1)[-2:])
    year_frame = leaguegamelog.LeagueGameLog(season=season).get_data_frames()
    
    return year_frame

frame = pd.DataFrame(get_team_for_year(2012)[0])
print(frame['TEAM_NAME'].unique())

['Los Angeles Lakers' 'Dallas Mavericks' 'Cleveland Cavaliers'
 'Boston Celtics' 'Miami Heat' 'Washington Wizards' 'Houston Rockets'
 'Los Angeles Clippers' 'Memphis Grizzlies' 'Toronto Raptors'
 'Indiana Pacers' 'Utah Jazz' 'Portland Trail Blazers' 'Phoenix Suns'
 'Golden State Warriors' 'Sacramento Kings' 'Chicago Bulls'
 'Philadelphia 76ers' 'Denver Nuggets' 'New Orleans Hornets'
 'San Antonio Spurs' 'Detroit Pistons' 'Oklahoma City Thunder'
 'Minnesota Timberwolves' 'Milwaukee Bucks' 'Orlando Magic'
 'New York Knicks' 'Charlotte Bobcats' 'Atlanta Hawks' 'Brooklyn Nets']


In [29]:
# TODO Find efficient weight paramater (ELO) to be used in linear regression
# TODO plot data, linear regression may not capture much data due to distribition
# TODO plot player and game data with various plots (list of plots on note)


In [123]:
players = pd.DataFrame(pd.read_csv("players.csv"))




# start with this function to find all players played in a seaosn. This will return a column of unique player IDs that played for a season
def get_players_for_season(season_start):
    file_location = "../player_season/player_season_data/season_year_start_player_data_" + str(season_start)
    season = pd.read_csv(file_location)
    players_id = season['PLAYER_ID'].drop_duplicates()
    
    return players_id


#use this to find a players ID. not needed but for convienience
def find_player_id(player_full_name):
    for i in range(0, len(players)):
        player = players.iloc[i]
        if player["full_name"] == player_full_name:
            return player['id']
        
# once you have that list of player ID use this function next and just input the season start you want, and keep calling this function and it will return the players stat
def get_player_stat_for_season(season_start, player_name = None):
    # check if player played for that season.
    # if not played return program with error
    
    file_location = "../player_season/player_season_data/season_year_start_player_data_" + str(season_start)
    season = pd.read_csv(file_location)
    return(season)



def get_stat_for_player(season, player_id):
     player_stat = season[season['PLAYER_ID'] == player_id]        
     return player_stat
             
# data frame object
# Lebron test

# this line of code shows you how to get lebrosn stat for the 2014 seaosn start
# player_season_stat = get_stat_for_player(get_player_stat_for_season(season_start="2014"),player_id=find_player_id("LeBron James"))


def total_function_for_players_stat_in_season(season_start_year):
    

    players = get_players_for_season(season_start_year)



    path = './player_data_for_' + str(season_start_year)
    box_score_path = './box_score_for_players_in_season' + str(season_start_year)
    players_stats_in_season = []

    # remove try block to prevent duplicates after
    try: 
        os.mkdir(path)
    except FileExistsError:
        pass
   

    # find all games in a season then create a csv file for the boxscores
    try: 
        os.mkdir(box_score_path)
    except FileExistsError:
        pass

    player_season_stat = get_player_stat_for_season(season_start=season_start_year)

    for player in player_season_stat['PLAYER_ID'].unique():
        stat = get_stat_for_player(player_season_stat, player_id=player)
        stat['NET_RANKING'] = np.NAN
        path_player = path + '/' + str(player) + '.csv'
        stat.to_csv(path_player)

    # count = 0

    # for game_id in player_season_stat['GAME_ID'].unique():
    #     game_id = '00' + str(game_id)
    #     box_score_score_for_game = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=game_id).get_data_frames()[0]
    #     time.sleep(0.2)
        
    #     print(count)
    #     count = count + 1
    #     box_score_file_path = box_score_path + '/' + game_id + '.csv'
    #     box_score_score_for_game.to_csv(box_score_file_path)


    result = pd.DataFrame()
    
    for player_file in os.listdir(path):
        path_player = path + '/'+ player_file
        plyr = pd.read_csv(path_player)
        result = pd.merge(plyr, box, on = ['PLAYER_ID','GAME_ID'])
    
        
        
        

total_function_for_players_stat_in_season('2014')








/var/folders/jm/cm59457511v80mtdqy5_hx7c0000gn/T/ipykernel_1529/3420195336.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stat['NET_RANKING'] = np.NAN
/var/folders/jm/cm59457511v80mtdqy5_hx7c0000gn/T/ipykernel_1529/3420195336.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stat['NET_RANKING'] = np.NAN
/var/folders/jm/cm59457511v80mtdqy5_hx7c0000gn/T/ipykernel_1529/3420195336.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

KeyError: 'NET_RANKING'

In [ ]:
# data preprocessing and cleaning and analysis

# for every player_stat_object 

# add column for team versed
# go through every row and after very 'vs' find the teamId and column (against team column)
# might not need cause data is already connected through gameID

# TODO Dealing with N/A values with old data
# TODO probably better of just deleting rows with N/A values 
# TODO maube estimate


# read from a player csv file or from a function
player_season_stat = pd.read_csv("lebron_test_file.csv")

player_season_stat = player_season_stat.drop('TEAM_NAME', axis=1)
player_season_stat = player_season_stat.drop('PLAYER_NAME', axis=1)
player_season_stat = player_season_stat.drop('NICKNAME', axis=1)
player_season_stat = player_season_stat.drop('TEAM_ABBREVIATION', axis=1)
player_season_stat = player_season_stat.drop('WNBA_FANTASY_PTS_RANK', axis=1)
player_season_stat = player_season_stat.drop('NBA_FANTASY_PTS_RANK', axis=1)
player_season_stat = player_season_stat.drop('NBA_FANTASY_PTS', axis=1)
player_season_stat = player_season_stat.drop('WNBA_FANTASY_PTS', axis=1)
# Maybe keep season year
player_season_stat = player_season_stat.drop('SEASON_YEAR', axis=1)
player_season_stat = player_season_stat.drop('Unnamed: 0.1', axis=1)
player_season_stat = player_season_stat.drop('Unnamed: 0', axis=1)
player_season_stat['GAME_DATE'] = pd.to_datetime(player_season_stat['GAME_DATE'])


# drop MIN_SEC since it is an object 
player_season_stat = player_season_stat.drop('MIN_SEC', axis=1)




def home_away_map(x):
    try:
        index = x.index('@')
        return 0
    except ValueError:
        return 1


from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
player_season_stat['WL'] = lb.fit_transform(player_season_stat['WL']) # check type, maybe convert to boolean or osemthing
player_season_stat['MATCHUP'] =  player_season_stat['MATCHUP'].map(home_away_map)
print(player_season_stat.info())
player_season_stat.to_csv('lebron_proccessed_test_file.csv')




# choose columsn to standardise.


# move Win lose column to first column.
# plot adn see 










In [ ]:
# PYTORCH
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

my_matrix = player_season_stat.corr(method="spearman").round(2)

#my_matrix.style.background_gradient(cmap="Greens")

#pd.plotting.scatter_matrix(my_matrix, figsize=(8,8))

my_mask = np.triu(np.ones_like(my_matrix,dtype=bool))

plt.figure(figsize=(100,80)) #width, height
sns.heatmap(my_matrix, cmap="Blues", vmin=0, vmax=1,
           annot=True, fmt="0.2f", square=True, mask=my_mask)

plt.savefig('./data_plots/lebron_corr_matrix_test.png')





